In [121]:
#Import relevant librarie
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import precision_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from numpy import nan
import random
import collections
import itertools
from sklearn import preprocessing
import xgboost as xgb
from collections import Counter

### Read the Feature file

In [122]:
data_path = "C:\\Users\\AH262855\\Desktop\\Nouveau dossier\\Data_ML\\perc_initialization\\features_train_data.csv"
data = pd.read_csv(data_path,index_col=0,encoding = "ISO-8859-1")
print(data.shape)
data.head()

(4000, 12)


,nb_functions_G2,max_connections,min_connections,mean_connections,variance_connections,mean_val_connect,std_val_connect,SAS_RCSL,PS_PAS,PS_SAS,PS_RCSL,DAS_PAS
case_study,,,,,,,,,,,,
F2000_C100_X0_0,539,322,64,162.523191,31.995076,892.359926,178.231743,21837,42806,41149,0,23327
F2000_C100_X0_1,552,306,55,162.884058,30.502524,894.579710,170.594639,22672,42044,40049,0,22546
F2000_C100_X0_2,529,317,44,157.903592,35.170098,868.816635,198.434141,19052,38128,34617,0,21565
F2000_C100_X0_3,549,302,49,157.783242,32.691580,868.520947,185.294720,22412,35674,36916,0,20543
F2000_C100_X0_4,567,299,29,158.922399,33.107080,875.190476,187.357934,22899,39454,36779,0,23380


### Read the output file
We have 5 intervals and the output of a study case is one of the means of these 5 intervals.
<br> These values are replaced by an integer between 0 and 4. Each integer represents one classe (one value).

In [123]:
sizes_path = "C:\\Users\\AH262855\\Desktop\\Nouveau dossier\\Data_ML\\perc_initialization\\best_perc_ini.csv"
output = pd.read_csv(sizes_path,index_col=0,encoding = "ISO-8859-1")
perc_ini = [0.05,0.15,0.25,0.35,0.45]
temp = np.zeros((len(output),len(perc_ini)))
y = np.zeros(len(output))
for i in range(0,len(output)):
    a = np.argwhere(output.values[i]==perc_ini)
    temp[i,a[0]] = 1
    y[i] = a[0] + 1
#y = temp
y.shape

(4000,)

### Number of samples in each class

In [124]:
classes = [1,2,3,4,5]
counts = np.zeros(len(classes ))
for i in range(0,len(classes )):
    counts[i] = len(np.argwhere(y==classes [i]))
counts /= len(y)
counts

array([0.032  , 0.1265 , 0.361  , 0.34575, 0.13475])

## Correlation between input and output

In [125]:
import seaborn as sns
df_1 = pd.DataFrame(data= output)
df = data.copy()
df['y'] =  df_1
#plt.figure(figsize=(5,5))
#sns.heatmap(df.corr())
corr = df.corr()
temp = corr['y']
temp
#temp.style.background_gradient(cmap='coolwarm').set_precision(2)

nb_functions_G2        -0.391405
max_connections         0.150528
min_connections         0.115390
mean_connections        0.133920
variance_connections    0.194717
mean_val_connect        0.133896
std_val_connect         0.194706
SAS_RCSL               -0.040568
PS_PAS                 -0.017904
PS_SAS                 -0.040431
PS_RCSL                -0.217614
DAS_PAS                -0.019785
y                       1.000000
Name: y, dtype: float64

### Split and normalization of the data

In [126]:
" Split the data"
X_train, X_val, y_train, y_val = train_test_split(data, y, test_size=0.1,random_state=5,stratify=y)

In [127]:
" normalization of the data"
mean = X_train.mean(axis=0)
stdev = X_train.std(axis=0)
X_train_normalize = (X_train - mean)/stdev
X_val_normalize =  (X_val - mean)/stdev
#X_test_normalize =  (X_test - mean)/stdev


In [131]:
print(confusion_matrix(y_val,y_pred_val))

[[ 0  7  4  1  1]
 [ 1 11 23 10  6]
 [ 1 12 91 34  6]
 [ 0  2 42 83 11]
 [ 0  0  5 27 22]]


## XGBoost Classifier

In [128]:
XGB = xgb.XGBClassifier(objective= 'multi:softmax', num_class =5)
param = {
    'max_depth': [4],
    'eta': [0.3],#np.arange(0.2,1,0.1), 
    'gamma': [0], 
    'lambda':np.arange(0.5,1,0.1),
    'alpha': np.arange(0,0.5,0.1), }
best_xgb = GridSearchCV(estimator=XGB,param_grid=param,n_jobs = 8,cv = 5)
best_xgb.fit(X_train_normalize, y_train)
print(best_xgb.best_params_)
y_pred_train = best_xgb.predict(X_train_normalize)
print("train_accuracy: ", accuracy_score(y_train,y_pred_train))
y_pred_val = best_xgb.predict(X_val_normalize)
print("val_accuracy: ",accuracy_score(y_val,y_pred_val))
print("confusion matrix of validation data")
print(confusion_matrix(y_val,y_pred_val))
target_names = ['0.05','0.15', '0.25', '0.35', '0.45']
print(classification_report(y_val, y_pred_val, target_names=target_names))
#print(confusion_matrix(y_train,y_pred_train))
matrix = confusion_matrix(y_val,y_pred_val)
matrix.diagonal()/matrix.sum(axis=1)

{'alpha': 0.4, 'eta': 0.3, 'gamma': 0, 'lambda': 0.7999999999999999, 'max_depth': 4}
train_accuracy:  0.88
val_accuracy:  0.5175
confusion matrix of validation data
[[ 0  7  4  1  1]
 [ 1 11 23 10  6]
 [ 1 12 91 34  6]
 [ 0  2 42 83 11]
 [ 0  0  5 27 22]]
              precision    recall  f1-score   support

        0.05       0.00      0.00      0.00        13
        0.15       0.34      0.22      0.27        51
        0.25       0.55      0.63      0.59       144
        0.35       0.54      0.60      0.57       138
        0.45       0.48      0.41      0.44        54

    accuracy                           0.52       400
   macro avg       0.38      0.37      0.37       400
weighted avg       0.49      0.52      0.50       400



array([0.        , 0.21568627, 0.63194444, 0.60144928, 0.40740741])

# Add random Data

In [114]:
df = pd.DataFrame(columns = data.columns) 
columns = data.columns
for column in  columns:
    maximum = max( data[column])
    minimum = min(data[column])
    df[column] = np.random.randint(minimum,maximum,1000)
  
add_random_data = data.copy()
add_random_data = add_random_data.append(df, ignore_index=False)
Y = np.random.choice(classes, 1000, p =counts)
y_random = y.copy()
y_random  = np.append(y_random ,Y)

### Split and normalization of the data

In [115]:
" Split the data"
X_train, X_val, y_train, y_val = train_test_split(add_random_data,y_random ,test_size=0.1,random_state=5,stratify=y_random )

In [116]:
" normalization of the data"
mean = X_train.mean(axis=0)
stdev = X_train.std(axis=0)
X_train_normalize = (X_train - mean)/stdev
X_val_normalize =  (X_val - mean)/stdev
#X_test_normalize =  (X_test - mean)/stdev

## XGBoost Classifier

In [119]:
XGB = xgb.XGBClassifier(objective= 'multi:softmax',num_class =5)
param = {
    'max_depth': [4],#np.arange(3,6,1),
    'eta': [0.3],#np.arange(0.2,0.5,0.1), 
    'gamma': [0], 
    'lambda':[0.8],#np.arange(0.7,1,0.1),
    'alpha': [0.4],}#np.arange(0,0.3,0.1), }
best_xgb = GridSearchCV(estimator=XGB,param_grid=param,n_jobs = 8,cv = 5)
best_xgb.fit(X_train_normalize, y_train)
print(best_xgb.best_params_)
y_pred_train = best_xgb.predict(X_train_normalize)
print("train_accuracy: ", accuracy_score(y_train,y_pred_train))
y_pred_val = best_xgb.predict(X_val_normalize)
print("val_accuracy: ",accuracy_score(y_val,y_pred_val))
print("confusion matrix of validation data")
print(confusion_matrix(y_val,y_pred_val))
target_names = ['0.05','0.15', '0.25', '0.35', '0.45']
print(classification_report(y_val, y_pred_val, target_names=target_names))
print(confusion_matrix(y_train,y_pred_train))
print(classification_report(y_train, y_pred_train, target_names=target_names))


{'alpha': 0.4, 'eta': 0.3, 'gamma': 0, 'lambda': 0.8, 'max_depth': 4}
train_accuracy:  0.8211111111111111
val_accuracy:  0.488
confusion matrix of validation data
[[  1   1   7   5   2]
 [  1   4  40  16   1]
 [  0   7 121  46   7]
 [  0   6  60  95  12]
 [  0   3  14  28  23]]
              precision    recall  f1-score   support

        0.05       0.50      0.06      0.11        16
        0.15       0.19      0.06      0.10        62
        0.25       0.50      0.67      0.57       181
        0.35       0.50      0.55      0.52       173
        0.45       0.51      0.34      0.41        68

    accuracy                           0.49       500
   macro avg       0.44      0.34      0.34       500
weighted avg       0.46      0.49      0.46       500

[[ 122    5   14    5    1]
 [   1  349  152   48    6]
 [   0   12 1427  167   17]
 [   0    5  216 1304   35]
 [   0    5   26   90  493]]
              precision    recall  f1-score   support

        0.05       0.99      0.83   